In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
from creation_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'data/D4_filt_10mino3.csv',index_col = 0)
print df_P[:5]

                       O3_ppb      UnixTime      e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                          
2014-07-13 02:51:00        39  1.405220e+09  80.666667  25.100000  53.866667   
2014-07-13 02:52:00        40  1.405220e+09  80.000000  25.066667  53.833333   
2014-07-13 02:53:00        40  1.405220e+09  79.750000  25.000000  53.925000   
2014-07-13 02:54:00        39  1.405220e+09  80.333333  24.933333  54.000000   
2014-07-13 02:55:00        38  1.405220e+09  80.000000  24.900000  54.033333   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-13 02:51:00                 107.47310                0             39   
2014-07-13 02:52:00                 107.34985                0             40   
2014-07-13 02:53:00                 107.22607                0             40   
2014-07-13 02:54:00               

In [19]:
df_P.drop(df_P.columns[[0,2]], axis=1, inplace=True)
print df_P[:5]

                           UnixTime       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                       
2014-07-13 02:51:00    1.405220e+09  25.100000  53.866667   
2014-07-13 02:52:00    1.405220e+09  25.066667  53.833333   
2014-07-13 02:53:00    1.405220e+09  25.000000  53.925000   
2014-07-13 02:54:00    1.405220e+09  24.933333  54.000000   
2014-07-13 02:55:00    1.405220e+09  24.900000  54.033333   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
2014-07-13 02:51:00                 107.47310                0             39   
2014-07-13 02:52:00                 107.34985                0             40   
2014-07-13 02:53:00                 107.22607                0             40   
2014-07-13 02:54:00                 107.10176                0             39   
2014-07-13 02:55:00                 106.97693                0             38   

    

###Call the slope functions and add them to the dataframe.

In [20]:
#ozone terms
df_P['O3_sqrt'] = np.sqrt(df_P['pod_o3_smooth'].astype(float))
df_P['O3_sq'] = df_P['pod_o3_smooth']**2
df_P['O3_cu'] = df_P['pod_o3_smooth']**3
df_P['ln_O3'] = np.log(df_P['pod_o3_smooth'])

#temp terms
df_P['temp_sqrt'] = np.sqrt(df_P['Temp'].astype(float))
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] =np.log(df_P['Temp'])

#rh terms
df_P['rh_sqrt'] = np.sqrt(df_P['Rh'].astype(float))
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])

#ozone/temp interactions
df_P['03_mult_temp'] = df_P['pod_o3_smooth']*df_P['Temp']
df_P['03_sq_mult_temp_sq'] = ((df_P['pod_o3_smooth'])**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['pod_o3_smooth'])

#ozone/rh interactions
df_P['03_mult_rh'] = df_P['pod_o3_smooth']*df_P['Rh']
df_P['03_sq_mult_rh_sq'] = (df_P['pod_o3_smooth']**2)*df_P['Rh']**2
df_P['ln_03_mult_ln_rh'] = np.log(df_P['Rh'])*np.log(df_P['pod_o3_smooth'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_sq_mult_rh_sq'] = (df_P['Temp']**2)*df_P['Rh']**2
df_P['ln_temp_mult_ln_rh'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])

#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['pod_o3_smooth']*df_P['Rh']*df_P['Temp']
df_P['03_sq_mult_rh_sq_&_temp_sq'] = (df_P['pod_o3_smooth']**2)*(df_P['Rh']**2)*df_P['Temp']**2
df_P['ln_03_mult_ln_rh_&_ln_temp'] = np.log(df_P['Temp'])*np.log(df_P['Rh'])*np.log(df_P['pod_o3_smooth'])

feature_list = ['pod_o3_smooth', 'ln_O3', 'Temp', 'ln_temp', 'Rh', 'ln_rh']

for i in range(0,len(feature_list)):
    make_func_caller_find_lag_integral(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 15, 90, 15, feature_list[i])
    make_func_caller_find_lag_integral(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lead_integral(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lag_slope(df_P, 5, 5, 15, feature_list[i])
    make_func_caller_find_lead_slope(df_P, 5, 5, 15, feature_list[i])

features = list(df_P.columns)
print features

['UnixTime', 'Temp', 'Rh', 'Zenith Angle [degrees]', 'days from start', 'ref_o3_smooth', 'pod_o3_smooth', 'O3_sqrt', 'O3_sq', 'O3_cu', 'ln_O3', 'temp_sqrt', 'temp_sq', 'temp_cu', 'ln_temp', 'rh_sqrt', 'rh_sq', 'rh_cu', 'ln_rh', '03_mult_temp', '03_sq_mult_temp_sq', 'ln_03_mult_ln_temp', '03_mult_rh', '03_sq_mult_rh_sq', 'ln_03_mult_ln_rh', 'temp_mult_rh', 'temp_sq_mult_rh_sq', 'ln_temp_mult_ln_rh', '03_mult_rh_&_temp', '03_sq_mult_rh_sq_&_temp_sq', 'ln_03_mult_ln_rh_&_ln_temp', 'pod_o3_smooth_int_lag_15', 'pod_o3_smooth_int_lag_30', 'pod_o3_smooth_int_lag_45', 'pod_o3_smooth_int_lag_60', 'pod_o3_smooth_int_lag_75', 'pod_o3_smooth_int_lag_90', 'pod_o3_smooth_int_lead_15', 'pod_o3_smooth_int_lead_30', 'pod_o3_smooth_int_lead_45', 'pod_o3_smooth_int_lead_60', 'pod_o3_smooth_int_lead_75', 'pod_o3_smooth_int_lead_90', 'diff', 'pod_o3_smooth_slope_lag_15', 'pod_o3_smooth_slope_lag_30', 'pod_o3_smooth_slope_lag_45', 'pod_o3_smooth_slope_lag_60', 'pod_o3_smooth_slope_lag_75', 'pod_o3_smooth_sl

In [21]:
#Delete the first and last 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[90:len(df_P['pod_o3_smooth'])-90]

In [22]:
#ozone/temp
df_P['o3_temp_int_lag_30'] = df_P['pod_o3_smooth_int_lag_30'] * df_P['Temp_int_lag_30'] 
df_P['o3_temp_slope_lag_30'] = df_P['pod_o3_smooth_slope_lag_30'] * df_P['Temp_slope_lag_30']
df_P['o3_temp_int_lag_60'] = df_P['pod_o3_smooth_int_lag_60'] * df_P['Temp_int_lag_60'] 
df_P['o3_temp_slope_lag_60'] = df_P['pod_o3_smooth_slope_lag_60'] * df_P['Temp_slope_lag_60'] 
df_P['o3_temp_int_lag_90'] = df_P['pod_o3_smooth_int_lag_90'] * df_P['Temp_int_lag_90'] 
df_P['o3_temp_slope_lag_90'] = df_P['pod_o3_smooth_slope_lag_90'] * df_P['Temp_slope_lag_90']
df_P['o3_temp_int_lag_5'] = df_P['pod_o3_smooth_int_lag_5'] * df_P['Temp_int_lag_5'] 
df_P['o3_temp_slope_lag_5'] = df_P['pod_o3_smooth_slope_lag_5'] * df_P['Temp_slope_lag_5']

df_P['ln_o3_temp_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_temp_int_lag_30'] 
df_P['ln_o3_temp_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_temp_slope_lag_30']
df_P['ln_o3_temp_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_temp_int_lag_60'] 
df_P['ln_o3_temp_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_temp_slope_lag_60'] 
df_P['ln_o3_temp_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_temp_int_lag_90'] 
df_P['ln_o3_temp_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_temp_slope_lag_90']
df_P['ln_o3_temp_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_temp_int_lag_5'] 
df_P['ln_o3_temp_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_temp_slope_lag_5']

df_P['o3_temp_int_lead_30'] = df_P['pod_o3_smooth_int_lead_30'] * df_P['Temp_int_lead_30'] 
df_P['o3_temp_slope_lead_30'] = df_P['pod_o3_smooth_slope_lead_30'] * df_P['Temp_slope_lead_30'] 
df_P['o3_temp_int_lead_60'] = df_P['pod_o3_smooth_int_lead_60'] * df_P['Temp_int_lead_60'] 
df_P['o3_temp_slope_lead_60'] = df_P['pod_o3_smooth_slope_lead_60'] * df_P['Temp_slope_lead_60'] 
df_P['o3_temp_int_lead_90'] = df_P['pod_o3_smooth_int_lead_90'] * df_P['Temp_int_lead_90'] 
df_P['o3_temp_slope_lead_90'] = df_P['pod_o3_smooth_slope_lead_90'] * df_P['Temp_slope_lead_90']
df_P['o3_temp_int_lead_5'] = df_P['pod_o3_smooth_int_lead_5'] * df_P['Temp_int_lead_5'] 
df_P['o3_temp_slope_lead_5'] = df_P['pod_o3_smooth_slope_lead_5'] * df_P['Temp_slope_lead_5']

df_P['ln_o3_temp_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_temp_int_lead_30'] 
df_P['ln_o3_temp_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_temp_slope_lead_30'] 
df_P['ln_o3_temp_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_temp_int_lead_60'] 
df_P['ln_o3_temp_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_temp_slope_lead_60'] 
df_P['ln_o3_temp_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_temp_int_lead_90'] 
df_P['ln_o3_temp_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_temp_slope_lead_90']
df_P['ln_o3_temp_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_temp_int_lead_5'] 
df_P['ln_o3_temp_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_temp_slope_lead_5']

#temp/rh
df_P['temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5']

df_P['ln_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5']

df_P['temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']
df_P['ln_temp_rh_int_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/rh
df_P['o3_rh_int_lag_30'] = df_P['pod_o3_smooth_int_lag_30'] * df_P['Rh_int_lag_30'] 
df_P['o3_rh_slope_lag_30'] = df_P['pod_o3_smooth_slope_lag_30'] * df_P['Rh_slope_lag_30'] 
df_P['o3_rh_int_lag_60'] = df_P['pod_o3_smooth_int_lag_60'] * df_P['Rh_int_lag_60'] 
df_P['o3_rh_slope_lag_60'] = df_P['pod_o3_smooth_slope_lag_60'] * df_P['Rh_slope_lag_60'] 
df_P['o3_rh_int_lag_90'] = df_P['pod_o3_smooth_int_lag_90'] * df_P['Rh_int_lag_90'] 
df_P['o3_rh_slope_lag_90'] = df_P['pod_o3_smooth_slope_lag_90'] * df_P['Rh_slope_lag_90'] 
df_P['o3_rh_int_lag_5'] = df_P['pod_o3_smooth_int_lag_5'] * df_P['Rh_int_lag_5'] 
df_P['o3_rh_slope_lag_5'] = df_P['pod_o3_smooth_slope_lag_5'] * df_P['Rh_slope_lag_5'] 

df_P['ln_o3_rh_int_lag_30'] = df_P['ln_O3_int_lag_30'] * df_P['ln_rh_int_lag_30'] 
df_P['ln_o3_rh_slope_lag_30'] = df_P['ln_O3_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] 
df_P['ln_o3_rh_int_lag_60'] = df_P['ln_O3_int_lag_60'] * df_P['ln_rh_int_lag_60'] 
df_P['ln_o3_rh_slope_lag_60'] = df_P['ln_O3_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] 
df_P['ln_o3_rh_int_lag_90'] = df_P['ln_O3_int_lag_90'] * df_P['ln_rh_int_lag_90'] 
df_P['ln_o3_rh_slope_lag_90'] = df_P['ln_O3_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] 
df_P['ln_o3_rh_int_lag_5'] = df_P['ln_O3_int_lag_5'] * df_P['ln_rh_int_lag_5'] 
df_P['ln_o3_rh_slope_lag_5'] = df_P['ln_O3_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] 

df_P['o3_rh_int_lead_30'] = df_P['pod_o3_smooth_int_lead_30'] * df_P['Rh_int_lead_30'] 
df_P['o3_rh_slope_lead_30'] = df_P['pod_o3_smooth_slope_lead_30'] * df_P['Rh_slope_lead_30'] 
df_P['o3_rh_int_lead_60'] = df_P['pod_o3_smooth_int_lead_60'] * df_P['Rh_int_lead_60'] 
df_P['o3_rh_slope_lead_60'] = df_P['pod_o3_smooth_slope_lead_60'] * df_P['Rh_slope_lead_60'] 
df_P['o3_rh_int_lead_90'] = df_P['pod_o3_smooth_int_lead_90'] * df_P['Rh_int_lead_90'] 
df_P['o3_rh_slope_lead_90'] = df_P['pod_o3_smooth_slope_lead_90'] * df_P['Rh_slope_lead_90']
df_P['o3_rh_int_lead_5'] = df_P['pod_o3_smooth_int_lead_5'] * df_P['Rh_int_lead_5'] 
df_P['o3_rh_slope_lead_5'] = df_P['pod_o3_smooth_slope_lead_5'] * df_P['Rh_slope_lead_5']

df_P['ln_o3_rh_int_lead_30'] = df_P['ln_O3_int_lead_30'] * df_P['ln_rh_int_lead_30'] 
df_P['ln_o3_rh_slope_lead_30'] = df_P['ln_O3_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] 
df_P['ln_o3_rh_int_lead_60'] = df_P['ln_O3_int_lead_60'] * df_P['ln_rh_int_lead_60'] 
df_P['ln_o3_rh_slope_lead_60'] = df_P['ln_O3_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] 
df_P['ln_o3_rh_int_lead_90'] = df_P['ln_O3_int_lead_90'] * df_P['ln_rh_int_lead_90'] 
df_P['ln_o3_rh_slope_lead_90'] = df_P['ln_O3_slope_lead_90'] * df_P['ln_rh_slope_lead_90']
df_P['ln_o3_rh_int_lead_5'] = df_P['ln_O3_int_lead_5'] * df_P['ln_rh_int_lead_5'] 
df_P['ln_o3_rh_slope_lead_5'] = df_P['ln_O3_slope_lead_5'] * df_P['ln_rh_slope_lead_5']

#ozone/temp/rh
df_P['o3_temp_rh_int_lag_30'] = df_P['Temp_int_lag_30'] * df_P['Rh_int_lag_30']*df_P['pod_o3_smooth_int_lag_30'] 
df_P['o3_temp_rh_slope_lag_30'] = df_P['Temp_slope_lag_30'] * df_P['Rh_slope_lag_30'] * df_P['pod_o3_smooth_slope_lag_30'] 
df_P['o3_temp_rh_int_lag_60'] = df_P['Temp_int_lag_60'] * df_P['Rh_int_lag_60']*df_P['pod_o3_smooth_int_lag_60'] 
df_P['o3_temp_rh_slope_lag_60'] = df_P['Temp_slope_lag_60'] * df_P['Rh_slope_lag_60'] * df_P['pod_o3_smooth_slope_lag_60'] 
df_P['o3_temp_rh_int_lag_90'] = df_P['Temp_int_lag_90'] * df_P['Rh_int_lag_90'] * df_P['pod_o3_smooth_int_lag_90']
df_P['o3_temp_rh_slope_lag_90'] = df_P['Temp_slope_lag_90'] * df_P['Rh_slope_lag_90'] * df_P['pod_o3_smooth_slope_lag_90'] 
df_P['o3_temp_rh_int_lag_5'] = df_P['Temp_int_lag_5'] * df_P['Rh_int_lag_5'] * df_P['pod_o3_smooth_int_lag_5']
df_P['o3_temp_rh_slope_lag_5'] = df_P['Temp_slope_lag_5'] * df_P['Rh_slope_lag_5'] * df_P['pod_o3_smooth_slope_lag_5'] 

df_P['ln_o3_temp_rh_int_lag_30'] = df_P['ln_temp_int_lag_30'] * df_P['ln_rh_int_lag_30']*df_P['ln_O3_int_lag_30'] 
df_P['ln_o3_temp_rh_slope_lag_30'] = df_P['ln_temp_slope_lag_30'] * df_P['ln_rh_slope_lag_30'] * df_P['ln_O3_slope_lag_30'] 
df_P['ln_o3_temp_rh_int_lag_60'] = df_P['ln_temp_int_lag_60'] * df_P['ln_rh_int_lag_60']*df_P['ln_O3_int_lag_60'] 
df_P['ln_o3_temp_rh_slope_lag_60'] = df_P['ln_temp_slope_lag_60'] * df_P['ln_rh_slope_lag_60'] * df_P['ln_O3_slope_lag_60'] 
df_P['ln_o3_temp_rh_int_lag_90'] = df_P['ln_temp_int_lag_90'] * df_P['ln_rh_int_lag_90'] * df_P['ln_O3_int_lag_90']
df_P['ln_o3_temp_rh_slope_lag_90'] = df_P['ln_temp_slope_lag_90'] * df_P['ln_rh_slope_lag_90'] * df_P['ln_O3_slope_lag_90'] 
df_P['ln_o3_temp_rh_int_lag_5'] = df_P['ln_temp_int_lag_5'] * df_P['ln_rh_int_lag_5'] * df_P['ln_O3_int_lag_5']
df_P['ln_o3_temp_rh_slope_lag_5'] = df_P['ln_temp_slope_lag_5'] * df_P['ln_rh_slope_lag_5'] * df_P['ln_O3_slope_lag_5'] 

df_P['o3_temp_rh_int_lead_30'] = df_P['Temp_int_lead_30'] * df_P['Rh_int_lead_30'] * df_P['pod_o3_smooth_int_lead_30'] 
df_P['o3_temp_rh_slope_lead_30'] = df_P['Temp_slope_lead_30'] * df_P['Rh_slope_lead_30'] * df_P['pod_o3_smooth_slope_lead_30']
df_P['o3_temp_rh_int_lead_60'] = df_P['Temp_int_lead_60'] * df_P['Rh_int_lead_60'] * df_P['pod_o3_smooth_int_lead_60'] 
df_P['o3_temp_rh_slope_lead_60'] = df_P['Temp_slope_lead_60'] * df_P['Rh_slope_lead_60'] * df_P['pod_o3_smooth_slope_lead_60'] 
df_P['o3_temp_rh_int_lead_90'] = df_P['Temp_int_lead_90'] * df_P['Rh_int_lead_90'] * df_P['pod_o3_smooth_int_lead_90']
df_P['o3_temp_rh_slope_lead_90'] = df_P['Temp_slope_lead_90'] * df_P['Rh_slope_lead_90'] * df_P['pod_o3_smooth_slope_lead_90']
df_P['o3_temp_rh_int_lead_5'] = df_P['Temp_int_lead_5'] * df_P['Rh_int_lead_5'] * df_P['pod_o3_smooth_int_lead_5']
df_P['o3_temp_rh_slope_lead_5'] = df_P['Temp_slope_lead_5'] * df_P['Rh_slope_lead_5'] * df_P['pod_o3_smooth_slope_lead_5']

df_P['ln_o3_temp_rh_int_lead_30'] = df_P['ln_temp_int_lead_30'] * df_P['ln_rh_int_lead_30'] * df_P['ln_O3_int_lead_30'] 
df_P['ln_o3_temp_rh_slope_lead_30'] = df_P['ln_temp_slope_lead_30'] * df_P['ln_rh_slope_lead_30'] * df_P['ln_O3_slope_lead_30']
df_P['ln_o3_temp_rh_int_lead_60'] = df_P['ln_temp_int_lead_60'] * df_P['ln_rh_int_lead_60'] * df_P['ln_O3_int_lead_60'] 
df_P['ln_o3_temp_rh_slope_lead_60'] = df_P['ln_temp_slope_lead_60'] * df_P['ln_rh_slope_lead_60'] * df_P['ln_O3_slope_lead_60']
df_P['ln_o3_temp_rh_int_lead_90'] = df_P['ln_temp_int_lead_90'] * df_P['ln_rh_int_lead_90'] * df_P['ln_O3_int_lead_90']
df_P['ln_o3_temp_rh_slope_lead_90'] = df_P['ln_temp_slope_lead_90'] * df_P['ln_rh_slope_lead_90'] * df_P['ln_O3_slope_lead_90']
df_P['ln_o3_temp_rh_int_lead_5'] = df_P['ln_temp_int_lead_5'] * df_P['ln_rh_int_lead_5'] * df_P['ln_O3_int_lead_5']
df_P['ln_o3_temp_rh_slope_lead_5'] = df_P['ln_temp_slope_lead_5'] * df_P['ln_rh_slope_lead_5'] * df_P['ln_O3_slope_lead_5']

##Save to a file. 

In [23]:
df_P.to_csv(path_or_buf = 'data/D4_smooth_features.csv')